In [1]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4

In [2]:
lambda_vector = [0.0, 0.0, 0.05]
# molecule string for H2O
molecule_string = """
0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""

# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}


In [3]:
calc = CQEDRHFCalculator(lambda_vector, molecule_string, psi4_options)
calc.calc_cqed_rhf_energy()


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Mon Jun 16 19:47:01 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

In [4]:
expected_h2o_e = -76.016355284146
assert np.isclose(expected_h2o_e, calc.cqed_rhf_energy, 1e-9, 1e-9)

In [5]:
calc.calc_scf_gradient(qed_wfn=True)


*** tstart() called on CHEM9QDFT72ALT
*** at Mon Jun 16 19:47:08 2025



array([[-3.69561367e-17,  9.32519913e-16, -6.01300796e-02],
       [-1.48146760e-17, -4.33923429e-02,  3.00650398e-02],
       [ 5.17708127e-17,  4.33923429e-02,  3.00650398e-02]])


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000     0.000000000000    -0.068516219320    15.994914619570
         H            0.000000000000    -0.790689573744     0.543701060715     1.007825032230
         H            0.000000000000     0.790689573744     0.543701060715     1.007825032230

  Nuclear repulsion =    8.80146556456340

In [ ]:
#mol = psi4.geometry(calc.molecule_string)
#psi4.set_options(calc.psi4_options)
#gradient = np.asarray(psi4.gradient("scf"))


In [ ]:
#print(calc.scf_grad -gradient)

In [6]:
calc.calc_quadrupole_gradient()
# last output

#[[ 1.32601432e-18  9.99294165e-19 -2.77573793e-03]
# [ 1.03219418e-19  2.17498212e-04  1.62681438e-03]
# [-1.42923374e-18 -2.17498212e-04  1.62681438e-03]]

[[-2.26303280e-19 -1.70226454e-18 -1.38786897e-03]
 [-1.49891387e-19  1.08749106e-04  8.13407191e-04]
 [ 3.76194667e-19 -1.08749106e-04  8.13407191e-04]]


In [7]:
calc.calc_numerical_gradient()


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Mon Jun 16 19:47:15 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

In [8]:
calc.numerical_scf_gradient.reshape(3,3)-calc.scf_grad

array([[ 1.88001515e-09, -2.63202209e-09,  5.36017339e-04],
       [ 7.89606350e-09,  3.43677045e-04, -2.68009421e-04],
       [-2.63202121e-09, -3.43681933e-04, -2.68008669e-04]])

In [9]:
print("O_DSE Analytic GRAD")
print(calc.numerical_o_dse_gradient.reshape(3,3))


print("O_DSE Numeric GRAD")
print(calc.o_dse_gradient.reshape(3,3))

O_DSE Analytic GRAD
[[-7.34380905e-13 -4.58988065e-13 -2.94970753e-03]
 [ 9.17976131e-14 -5.51459309e-04  1.71379917e-03]
 [-1.23926778e-12  5.51459311e-04  1.71379917e-03]]
O_DSE Numeric GRAD
[[-2.26303280e-19 -1.70226454e-18 -1.38786897e-03]
 [-1.49891387e-19  1.08749106e-04  8.13407191e-04]
 [ 3.76194667e-19 -1.08749106e-04  8.13407191e-04]]


In [10]:
calc.calc_dipole_dipole_gradient()

In [11]:
print("K_DSE GRAD")
print(calc.numerical_K_dse_gradient.reshape(3,3))

print("ANALYTIC K_DSE GRAD")
print(calc.K_dse_gradient.reshape(3,3))




K_DSE GRAD
[[ 2.06544629e-13  2.63918138e-13  7.08281359e-04]
 [-1.14747016e-13  1.01756985e-05 -5.93086080e-04]
 [ 4.01614557e-13 -1.01756990e-05 -5.93086080e-04]]
ANALYTIC K_DSE GRAD
[[-4.33758000e-19  6.39290074e-09  1.07036548e-03]
 [ 1.88553948e-19 -4.15106205e-04 -7.74111366e-04]
 [ 2.45204052e-19  4.15099812e-04 -7.74107364e-04]]


In [12]:

print("E GRAD")
print(calc.numerical_energy_gradient.reshape(3,3))

sum_grad = calc.scf_grad + calc.numerical_o_dse_gradient.reshape(3,3) + calc.numerical_K_dse_gradient.reshape(3,3)

print("SUM GRAD")
print(sum_grad)

E GRAD
[[ 3.76003023e-09 -2.25601814e-09 -6.18354847e-02]
 [ 6.76805442e-09 -4.35899478e-02  3.09177412e-02]
 [ 3.76003023e-10  4.35899447e-02  3.09177435e-02]]
SUM GRAD
[[-5.27873231e-13 -1.94137408e-13 -6.23715057e-02]
 [-2.29642179e-14 -4.39336265e-02  3.11857529e-02]
 [-8.37601448e-13  4.39336265e-02  3.11857529e-02]]


In [ ]:
total_gradient = calc.scf_grad + calc.K_dse_gradient.reshape(3,3) + calc.o_dse_gradient.reshape(3,3)
print(total_gradient)

# The numerical gradient for water in a cc-pVDZ basis set with this geometry and lambda = 0, 0, 0.05 is:

[[ 2.25601351e-10 -2.63201576e-10 -6.18353559e-02]

 [ 7.52004504e-11 -4.35898538e-02  3.09176777e-02]
 
 [ 0.00000000e+00  4.35898541e-02  3.09176786e-02]]

In [ ]:
_expected_gradient = np.array(
[[ 2.25601351e-10, -2.63201576e-10, -6.18353559e-02],
 [ 7.52004504e-11, -4.35898538e-02,  3.09176777e-02],
 [ 0.00000000e+00,  4.35898541e-02,  3.09176786e-02]])

print(_expected_gradient-calc.numerical_energy_gradient.reshape(3,3))